# SCC-DFTB repulsive potential generation (2-body)

Input: DFT_DB.db DFTB_DB.db 

Output: CCS parameters to be used in conjunction with the SK-tables generated in gen_SK.ipynb and CCS_DB.db 



In [ ]:
PARAUTOMATIK_PATH='WRITE-Your-PATH-PARAUTOMATIK-Here'   # Add your installation path here

In [ ]:
import os, sys, warnings
sys.path.append(PARAUTOMATIK_PATH+'/pgm/CCS/')
sys.path.append(PARAUTOMATIK_PATH+'/pgm/python_scripts')
import json
import itertools as it
from collections import OrderedDict, defaultdict
import numpy as np
from ase import Atoms
from ase import io
from ase.calculators.neighborlist import NeighborList
import ase.db as db
from tqdm import tqdm 
from ccs.scripts.ccs_fetch import ccs_fetch as ccs_fetch
from ccs.fitting.main import twp_fit as ccs_fit

from ccs.common.exceptions import ScriptError 
from ccs.ase_caclulator.ccs_ase_calculator import CCS

import matplotlib.pyplot as plt
import seaborn as sns


base_dir=os.getcwd()
os.chdir(base_dir)
print('Base dir is:', base_dir)
warnings.filterwarnings('ignore')


In [ ]:
#generate data generates trainingset in the format for CCS (structures.json)
cwd = os.getcwd()
if not os.path.isdir(cwd+"/CCS/"):
    os.mkdir(cwd+"/CCS/")
os.chdir(cwd+"/CCS/")


ccs_fetch(mode='DFTB', R_c=6.0, Ns='all', DFT_DB='../DFT_DB.db', DFTB_DB='../DFTB_DB.db')

os.chdir(base_dir) 

In [ ]:
#DEFINE INPUT
os.chdir(cwd+"/CCS/")

input={
    "General": {
        "interface": "DFTB"
    },
    "Twobody": {
        "C-O": {
            "Rcut": 6.0,
            "Resolution": 0.05,
            "Swtype": "rep"
               }
    }
}
#SAVE TO FILE
with open('input.json', 'w') as f:
    json.dump(input, f, indent=8)
    
#RUN FIT
ccs_fit("input.json")
os.chdir(base_dir)

In [ ]:
os.chdir(base_dir+"/CCS/")
class Args:
    cmdlineargs=None
    input = 'input.json'

args=Args()
fit_ccs(args)
os.chdir(base_dir) 

# Analysis

In [7]:
os.chdir(base_dir)

struct=db.connect("DFTB_DB.db")
db_ccs=db.connect("CCS_DB.db")

os.chdir(base_dir+"/CCS/")
with open('CCS_params.json', 'r') as f:
    CCS_params = json.load(f)

for row in tqdm(struct.select(), total=len(struct)):  
    structure=row.toatoms()
    structure.calc=CCS(CCS_params=CCS_params)
    structure.get_potential_energy()
    structure.get_forces()
    db_ccs.write(structure,key=row.key,CCS=True)

os.chdir(base_dir)

In [ ]:
#Read Data
import matplotlib.pyplot as plt
import seaborn as sns
dbname_DFT='DFT_DB.db'
dbname_DFTB='DFTB_DB.db'
dbname_CCS='CCS_DB.db'
db_dftb=db.connect(dbname_DFTB)
db_ccs=db.connect(dbname_CCS)
db_dft=db.connect(dbname_DFT)
epred=[];edft=[];edftb=[];eccs=[];
for row in tqdm(db_dftb.select(), total=len(db_dftb)):
    structure=row.toatoms()
    natoms=len(structure.get_atomic_numbers())
    id=str(row.key)
    edftb.append(db_dftb.get('key='+id).energy/natoms)
    eccs.append(db_ccs.get('key='+id).energy/natoms)
    edft.append(db_dft.get('key='+id).energy/natoms) 
    edftb_css.append(db_dftb.get('key='+id).energy/natoms + db_ccs.get('key='+id).energy/natoms)


In [ ]:
#plot Energies
plt.figure(figsize = (10,10))
sns.scatterplot(edft, edftb_ccs, s=50, alpha=0.8, color='r')

plt.plot([np.min(edft),np.max(edft)], [np.min(edft),np.max(edft)], 'k-', lw=2.5)
plt.legend(labels=["DFTB+CCS"], fontsize=30 )
plt.xlabel('E$_{DFT}$/atom (eV)',fontsize=30)
plt.ylabel('E$_{DFTB+CCS}$',fontsize=30)
plt.xlim(np.min(edft),np.max(edft))
plt.ylim(np.min(edft),np.max(edft))
plt.savefig('DFTvsDFTB+CCS.png')
plt.show()
